In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
class Value():
    def __init__(self, data, _children=(), _op = '', label=''):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self.grad = 0.0     # x.grad means derivative of loss fn (final node) wrt x
        self._backward = lambda: None # it is defined None function as default in case it is used for a leaf node

    def __repr__(self):
        return f'Value {self.label}(data = {self.data})'

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            '''
            if  x = y + z (think of it in terms of graph, will be easier)
            then gradient of x wrt y or z will be 1, i.e.,
            gradient of an addition operator propagates to its components, 
            therefore gradient of loss fn wrt components (y or z) will be
            equal to gradient of loss fn wrt x
            '''
            self.grad += 1.0 * out.grad      #since it is backpropagation, we already know gradient of loss fn wrt out
            other.grad += 1.0 * out.grad    # += because gradients accumulate, helful when a same node is used multiple times

        out._backward = _backward       #since self._backward is already defined as a function, therefore _backward not _backward()
        return out
    
    def __radd__(self, other):
        '''python redirects addition to radd when it sees 
        it can't handle addition in one order but can perform it in reverse order'''
        return self + other

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            '''
            if  x = y * z (think of it in terms of graph, will be easier)
            then gradient of x wrt y and z will be z and y respectively, 
            therefore gradient of loss fn wrt components y or z will be
            equal to gradient of loss fn wrt x * z or gradient of loss fn wrt x * y,
            respectively
            '''
            self.grad += other.data * out.grad      
            other.grad += self.data * out.grad

        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        '''python redirects multiplication to rmul when it sees 
        it can't handle multiplication in one order but can perform it in reverse order'''
        return self * other

    def exp(self): #e^self
        out = Value(math.exp(self.data), (self, ), _op = 'exp')

        def _backward():
            self.grad += out.data * out.grad

        out._backward = _backward
        return out

    def __pow__(self, other): #self**other
        assert isinstance(other, (int, float)), 'Power operator only supports int or float'

        out = Value(self.data**other, (self, ), _op = f'**{other}')

        def _backward():
            self.grad += other * self.data**(other-1) * out.grad
        out._backward = _backward
        return out

    def __truediv__(self, other): #self/other
        return self * other**-1

    def __neg__(self):  #-self
        return self * -1 

    def __sub__(self, other):   #self - other
        return self + (-other)

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward():
            self.grad += (1 - t**2)*out.grad
        out._backward = _backward
        return out
    
    def backward(self):
        self.grad = 1.0

        # using topological sort to arrange nodes in topological order
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        print('Gradients before backprop')
        for node in topo:
            print(node, node.grad)

        for node in reversed(topo):
            node._backward()

        print('\nGradients after backprop')
        for node in topo:
            print(node, node.grad)

## Using tanh fuunction

### A workflow where a, b and c are three leaf nodes, d and e are intermediary results and f is the final output

In [126]:
a = Value(2, label='a')
b = Value(-3, label='b')
c = Value(10.0, label='c')
d = a*b; d.label = 'd'
e = d + c; e.label='e'
f = e.tanh(); f.label='f'
print(d, e, f)

Value d(data = -6) Value e(data = 4.0) Value f(data = 0.999329299739067)


In [127]:
f._prev

{Value e(data = 4.0)}

In [128]:
f._op

'tanh'

### Manually calling the backward function to check if the derivatives are flowing correctly

In [116]:
f.grad = 1 # this has to be set since f is the output node and f.grad means derivative of output node f wrt f

In [117]:
f._backward()

In [118]:
print(e.grad)

0.0013409506830258655


In [119]:
e._backward()

In [120]:
print(d.grad, c.grad)

0.0013409506830258655 0.0013409506830258655


In [121]:
d._backward()

In [122]:
print(a.grad, b.grad)

-0.0040228520490775965 0.002681901366051731


In [123]:
a._backward()
print(a.grad)   #since a is a leaf node, a._backward() returns None function

-0.0040228520490775965


### Automating backpropagation

#### Topological Sort
All the arrows go in a single direction (left to right).
Required because we want to automate gradient calculation (calling backward function) and this should occur in order going from back to front.

In [129]:
f.backward()

Gradients before backprop
Value c(data = 10.0) 0.0
Value b(data = -3) 0.0
Value a(data = 2) 0.0
Value d(data = -6) 0.0
Value e(data = 4.0) 0.0
Value f(data = 0.999329299739067) 1.0

Gradients after backprop
Value c(data = 10.0) 0.0013409506830258655
Value b(data = -3) 0.002681901366051731
Value a(data = 2) -0.0040228520490775965
Value d(data = -6) 0.0013409506830258655
Value e(data = 4.0) 0.0013409506830258655
Value f(data = 0.999329299739067) 1.0


### Correcting backpropagation code

#### Before Correction

If a node is used multiple times as shown in example below, the code where accumulation of gradients is not taken into consideration i.e., only **`=`** is used like `self.grad =` won't work correctly. **`=`** should be replaced by **`+=`** to accumulate gradients.

In [4]:
n = Value(4.0, label='n')
m = n + n; m.label = 'm'
print(m, n)

Value m(data = 8.0) Value n(data = 4.0)


In [5]:
m._prev

{Value n(data = 4.0)}

In [6]:
print(n.grad, m.grad)

0.0 0.0


In [8]:
m.grad = 1.0
print(n.grad, m.grad)

0.0 1.0


In [10]:
m.backward()

Gradients before backprop
Value n(data = 4.0) 0.0
Value m(data = 8.0) 1.0

Gradients after backprop
Value n(data = 4.0) 1.0
Value m(data = 8.0) 1.0


#### After Correction

In [13]:
n = Value(4.0, label='n')
m = n + n; m.label = 'm'
print(m, n)

Value m(data = 8.0) Value n(data = 4.0)


In [14]:
m._prev

{Value n(data = 4.0)}

In [15]:
print(n.grad, m.grad)

0.0 0.0


In [16]:
m.grad = 1.0
print(n.grad, m.grad)

0.0 1.0


In [17]:
m.backward()

Gradients before backprop
Value n(data = 4.0) 0.0
Value m(data = 8.0) 1.0

Gradients after backprop
Value n(data = 4.0) 2.0
Value m(data = 8.0) 1.0


## Breaking tanh into components

In [3]:
a = Value(2, label='a')
b = Value(-3, label='b')
c = Value(10.0, label='c')
d = a*b; d.label = 'd'
e = d + c; e.label='e'
f = (2*e).exp(); f.label='f'
g = (f - 1)/(f + 1); g.label='g'
print(d, e, f, g)

Value d(data = -6) Value e(data = 4.0) Value f(data = 2980.9579870417283) Value g(data = 0.999329299739067)


In [9]:
g.grad = 1.0

In [ ]:
g.backward() #we see similar gradients as when tanh was used directly for the output node

Gradients before backprop
Value (data = 2) 0.0
Value c(data = 10.0) 0.0
Value a(data = 2) 0.0
Value b(data = -3) 0.0
Value d(data = -6) 0.0
Value e(data = 4.0) 0.0
Value (data = 8.0) 0.0
Value f(data = 2980.9579870417283) 0.0
Value (data = -1) 0.0
Value (data = 2979.9579870417283) 0.0
Value (data = 1) 0.0
Value (data = 2981.9579870417283) 0.0
Value (data = 0.0003353501304664781) 0.0
Value g(data = 0.999329299739067) 1.0

Gradients after backprop
Value (data = 2) 0.00268190136605213
Value c(data = 10.0) 0.001340950683026065
Value a(data = 2) -0.004022852049078195
Value b(data = -3) 0.00268190136605213
Value d(data = -6) 0.001340950683026065
Value e(data = 4.0) 0.001340950683026065
Value (data = 8.0) 0.0006704753415130325
Value f(data = 2980.9579870417283) 2.2491942000779598e-07
Value (data = -1) 0.0003353501304664781
Value (data = 2979.9579870417283) 0.0003353501304664781
Value (data = 1) -0.0003351252110464703
Value (data = 2981.9579870417283) -0.0003351252110464703
Value (data = 0.000